# Data Preparation

In [2]:
#Main
import pandas as pd
import numpy as np
from math import floor
import time
import pickle
from tqdm import tqdm
from collections import defaultdict# import warnings filter
from pandas.tseries.offsets import DateOffset

### Data download

In [3]:
LOAD = "/Users/marinacamacho/Desktop/Master_I/Raw_Data"
DUMP = "Processed"
DIS = "Depression"
MOD = "Models"

In [4]:
DropCols = [#"Ethnic", "Skin_colour",#"Assessment_centre",#"Sex",
            #"Recurrent_moderate_depression","Recurrent_severe_depression", "Single_depression_episode",
            "Depressed_2weeks", "Ever_depressed_1week", "Longest_depression", "Number_depression",
            #'APOA', 'APOB','Cholesterol', 'CRP', 'Glucose', 'HbA1c', 'HDL', 'IGF_1',
            #'Testosterone', 'Triglyceride', 
            'Seen_doctor', 'Seen_sychiatrist', 'Body_fat-free_mass','Body_fat_percentage',]

In [5]:
# Exposome data
df_expo = pd.read_csv(f'{LOAD}/Time_0/all_exposome_0.csv', usecols=lambda col: col not in DropCols)
df_expo = df_expo.set_index('f.eid')
df_expo.index.name = 'eid'
df_expo = df_expo.sort_index(axis=1)
df_expo2 = pd.read_csv(f'{LOAD}/Time_1/all_exposome_1.csv', usecols=lambda col: col not in DropCols)
df_expo2 = df_expo2.set_index('f.eid')
df_expo2 = df_expo2.sort_index(axis=1)
df_expo2.index.name = 'eid'

In [6]:
list(df_expo.columns)

['A/AS',
 'APOA',
 'APOB',
 'Able_to_confide',
 'Able_to_pay_rent/mortgage',
 'Abused_family_child',
 'Accident_life-threatening',
 'Active_than_usual',
 'Adopted_child',
 'Adult_education_class',
 'Age',
 'Age_full_education',
 'All_of_the_above',
 'Alzheimer_disease/dementia_F',
 'Alzheimer_disease/dementia_M',
 'Alzheimer_disease/dementia_S',
 'Assessment_centre',
 'Avoided_activities_1month',
 'BMI',
 'Back_pain',
 'Beef_intake',
 'Belittlement_partner',
 'Bipolar_II_Disorder',
 'Bipolar_I_Disorder',
 'Bipolar_disorder_status',
 'Bipolar_status',
 'Biscuit_cereal',
 'Bleeding_gums',
 'Bowel_cancer_F',
 'Bowel_cancer_M',
 'Bowel_cancer_S',
 'Bran_cereal',
 'Bread_intake',
 'Breast_cancer_F',
 'Breast_cancer_M',
 'Breast_cancer_S',
 'Breastfed_baby',
 'Brown',
 'Butter/spreadable_butter',
 'CRP',
 'CSE',
 'Cancer_age_first',
 'Cancer_diagnosed_by_doctor',
 'Care_home',
 'Cereal_intake',
 'Cheese_intake',
 'Cholesterol',
 'Chronic_bronchitis/emphysema_F',
 'Chronic_bronchitis/emphysem

In [7]:
df_expo.shape, df_expo2.shape

((502394, 248), (502394, 248))

In [8]:
#df_expo.dropna(subset=['Ethnic', 'Skin_colour', 'Sex'], inplace=True)
#df_expo2.dropna(subset=['Ethnic', 'Skin_colour', 'Sex'], inplace=True)

#df_expo = df_expo.sort_values(by=['eid']).reset_index(drop=True)
#df_expo2 = df_expo2.sort_values(by=['eid']).reset_index(drop=True)

In [9]:
output = 'Depression'

data = pd.read_csv("/Users/marinacamacho/Desktop/Master_I/Participants_Selection/PREDICTIVE_VISIT_DEPRESSION.csv",delimiter="\,")
#data.rename(columns={output: 'target'}, inplace=True)
data = data.set_index('eid')
data = data.sort_index(axis=1)

data2 = pd.read_csv("/Users/marinacamacho/Desktop/Master_I/Participants_Selection/PREDICTIVE_VISIT_DEPRESSION.csv",delimiter="\,")
#data2.rename(columns={output: 'target'}, inplace=True)
data2 = data2.set_index('eid')
data2 = data2.sort_index(axis=1)

healthy = pd.read_csv("/Users/marinacamacho/Desktop/Master_I/Participants_Selection/Healthy_Participants.csv",delimiter="\,")
healthy = healthy.set_index('eid')
healthy = healthy.sort_index(axis=1)

dates = pd.read_csv("/Users/marinacamacho/Desktop/Master_I/Participants_Selection/Date_Centre.csv",delimiter="\,")
dates = dates.set_index('eid')
dates = dates.sort_index(axis=1)
healthy = healthy.merge(dates, left_index=True, right_index=True, how='inner')
#healthy = healthy.set_index('eid')
#healthy = healthy.sort_index(axis=1)

/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [10]:
pd.DataFrame(df_expo['Sex']).to_csv(r'Sex0.csv', index = True, header=True)
pd.DataFrame(df_expo2['Sex']).to_csv(r'Sex1.csv', index = True, header=True)

In [11]:
healthy.head(1)

,date_centre_0,date_centre_1
eid,,
1000091,2009-04-17,NaN


In [12]:
data.head(5)

,CVD,Depression,Diabetes,visit0,visit1
eid,,,,,
1000028,1997-12-12,NaN,NaN,2009-10-26,NaN
1000159,NaN,2009-10-08,NaN,2008-04-01,NaN
1000648,2002-12-17,NaN,NaN,2008-02-02,NaN
1000813,2004-10-27,NaN,NaN,2008-12-04,NaN
1000880,2008-01-17,NaN,NaN,2010-02-24,NaN


In [13]:
datas = [data,data2]

for dataset in datas:
# convert your columns to datetime if they aren't already
    for column in list(dataset.columns):
        dataset[column] = pd.to_datetime(dataset[column])
        
    dataset['PastDiabetes'] = dataset.apply(lambda row: (pd.notnull(row['Diabetes'])) & 
                                       (row['Diabetes'] < row['visit0'] - DateOffset(years=1))
                                       , axis=1).astype(int)
    dataset['PastCVD'] = dataset.apply(lambda row: (pd.notnull(row['CVD'])) & 
                                       (row['CVD'] < row['visit0'] - DateOffset(years=1))
                                       , axis=1).astype(int)
    
    dataset['target'] = dataset.apply(lambda row: (pd.notnull(row['Depression'])) & 
                                       (row['Depression'] >= row['visit0'] + DateOffset(years=1))
                                       , axis=1).astype(int)

In [14]:
data.head(5)

,CVD,Depression,Diabetes,visit0,visit1,PastDiabetes,PastCVD,target
eid,,,,,,,,
1000028,1997-12-12,NaT,NaT,2009-10-26,NaT,0,1,0
1000159,NaT,2009-10-08,NaT,2008-04-01,NaT,0,0,1
1000648,2002-12-17,NaT,NaT,2008-02-02,NaT,0,1,0
1000813,2004-10-27,NaT,NaT,2008-12-04,NaT,0,1,0
1000880,2008-01-17,NaT,NaT,2010-02-24,NaT,0,1,0


In [15]:
data.shape, data.dropna(subset=['visit0']).shape, data.dropna(subset=['visit1']).shape

((47725, 8), (47725, 8), (1549, 8))

In [16]:
data['PastCVD'].value_counts()

0    26744
1    20981
Name: PastCVD, dtype: int64

In [17]:
data['PastDiabetes'].value_counts()

0    38606
1     9119
Name: PastDiabetes, dtype: int64

In [18]:
data['target'].value_counts()

0    25121
1    22604
Name: target, dtype: int64

In [19]:
healthy['CVD'] = None
healthy['Depression'] = None
healthy['Diabetes'] = None
healthy['visit0'] = healthy['date_centre_0']
healthy['visit1'] = healthy['date_centre_1']
del healthy['date_centre_0']
del healthy['date_centre_1']
for column in list(healthy.columns):
    healthy[column] = pd.to_datetime(healthy[column])
healthy['PastDiabetes'] = 0
healthy['PastCVD'] = 0
healthy['target'] = 0

In [20]:
data.head(1)

,CVD,Depression,Diabetes,visit0,visit1,PastDiabetes,PastCVD,target
eid,,,,,,,,
1000028,1997-12-12,NaT,NaT,2009-10-26,NaT,0,1,0


In [21]:
healthy.head(1)

,CVD,Depression,Diabetes,visit0,visit1,PastDiabetes,PastCVD,target
eid,,,,,,,,
1000091,NaT,NaT,NaT,2009-04-17,NaT,0,0,0


In [22]:
data_final = pd.concat([data,healthy],axis=0).sort_index(axis=1).sort_index(axis=0)

In [23]:
data_final.to_csv(r'data_final_outputs.csv', index = False, header=True)

In [24]:
data_final_1 = data_final.dropna(subset=['visit1']).sort_values(by=['eid'])
data_final_0 = data_final[~data_final.index.isin(data_final_1.index)].sort_values(by=['eid'])

In [25]:
data_final.shape, data_final_0.shape, data_final_1.shape, data_final_1.shape[0]==data_final.shape[0]-data_final_0.shape[0]

((113825, 8), (109727, 8), (4098, 8), True)

In [26]:
df_expo_0 = df_expo[df_expo.index.isin(data_final_0.index)].sort_values(by=['eid'])
df_expo_1 = df_expo2[df_expo2.index.isin(data_final_1.index)].sort_values(by=['eid'])

data_final_0 = data_final_0[data_final_0.index.isin(df_expo_0.index)].sort_values(by=['eid'])
data_final_1 = data_final_1[data_final_1.index.isin(df_expo_1.index)].sort_values(by=['eid'])

In [27]:
data_final_0.to_csv(r'data_final_outputs0.csv', index = False, header=True)
data_final_1.to_csv(r'data_final_outputs1.csv', index = False, header=True)

In [28]:
data_final_0['target'].value_counts()

0    87833
1    21870
Name: target, dtype: int64

In [29]:
data_final_1['target'].value_counts()

0    3370
1     728
Name: target, dtype: int64

In [30]:
df_expo_0.shape,df_expo_1.shape

((109703, 248), (4098, 248))

In [31]:
data_final_0.shape,data_final_1.shape

((109703, 8), (4098, 8))

In [32]:
features = ["Recurrent_moderate_depression", "Recurrent_severe_depression", "Single_depression_episode"]

for feature in features:
    # Identify the rows to drop for df_expo_0 and data_final_0
    mask_0 = (df_expo_0[feature] == 1) & (data_final_0['target'] == 0)
    rows_to_drop_0 = df_expo_0[mask_0].index
    df_expo_0.drop(rows_to_drop_0, inplace=True)
    data_final_0.drop(rows_to_drop_0, inplace=True)

    # Identify the rows to drop for df_expo_1 and data_final_1
    mask_1 = (df_expo_1[feature] == 1) & (data_final_1['target'] == 0)
    rows_to_drop_1 = df_expo_1[mask_1].index
    df_expo_1.drop(rows_to_drop_1, inplace=True)
    data_final_1.drop(rows_to_drop_1, inplace=True)
    
    del df_expo_0[feature]
    del df_expo_1[feature]

In [33]:
df_expo_0.shape,df_expo_1.shape

((105188, 245), (3969, 245))

In [34]:
data_final_0.shape,data_final_1.shape

((105188, 8), (3969, 8))

In [35]:
data_final_0['target'].value_counts()

0    83318
1    21870
Name: target, dtype: int64

In [36]:
data_final_1['target'].value_counts()

0    3241
1     728
Name: target, dtype: int64

In [37]:
df_expo_1['Assessment_centre'].value_counts()

11024.0    3969
Name: Assessment_centre, dtype: int64

In [38]:
df_expo_0['Assessment_centre'].value_counts()

11011.0    9667
11010.0    8984
11013.0    7575
11009.0    7244
11007.0    6943
11018.0    6573
11016.0    6096
11020.0    5633
11021.0    5216
11014.0    5194
11008.0    5070
11003.0    4753
11005.0    4401
11017.0    4351
11006.0    3962
11004.0    3520
11002.0    3230
11012.0    3186
11001.0    2224
10003.0     720
11022.0     515
11023.0     130
Name: Assessment_centre, dtype: int64

In [39]:
data = df_expo_0['Assessment_centre'].value_counts()

names = {
    11012.0: "Barts",
    11021.0: "Birmingham",
    11011.0: "Bristol",
    11008.0: "Bury",
    11003.0: "Cardiff",
    11024.0: "Cheadle",
    11020.0: "Croydon",
    11005.0: "Edinburgh",
    11004.0: "Glasgow",
    11018.0: "Hounslow",
    11010.0: "Leeds",
    11016.0: "Liverpool",
    11001.0: "Manchester",
    11017.0: "Middlesborough",
    11009.0: "Newcastle",
    11013.0: "Nottingham",
    11002.0: "Oxford",
    11007.0: "Reading",
    11014.0: "Sheffield",
    10003.0: "Stockport",
    11006.0: "Stoke",
    11022.0: "Swansea",
    11023.0: "Wrexham"
}


df = pd.DataFrame(list(data.items()), columns=["Centre", "Value"])
df['Centre'] = df['Centre'].replace(names)

In [40]:
def find_closest_to_30_percent(values, centres, total):
    target = int(0.3 * total)
    n = len(values)
    
    # Create a DP table
    dp = [[False for _ in range(target + 1)] for _ in range(n + 1)]
    
    # There is always a solution with 0 items to get sum 0
    for i in range(n + 1):
        dp[i][0] = True
        
    for i in range(1, n + 1):
        dp[i][values[i-1]] = True
        for j in range(1, target + 1):
            if dp[i-1][j] or (j >= values[i-1] and dp[i-1][j-values[i-1]]):
                dp[i][j] = True
                
    # Find the closest value to target
    for j in range(target, -1, -1):
        if dp[n][j]:
            closest_value = j
            break
            
    # Backtrack to find the centres
    selected_centres = []
    w = closest_value
    for i in range(n, 0, -1):
        if not dp[i-1][w]:
            selected_centres.append(centres[i-1])
            w -= values[i-1]

    return closest_value, selected_centres

values = df["Value"].tolist()
centres = df["Centre"].tolist()
total_value = sum(values)

closest_value, selected_centres = find_closest_to_30_percent(values, centres, total_value)

print(f"Closest value to 30% of total: {closest_value}")
print("Selected centres:", selected_centres)


Closest value to 30% of total: 31556
Selected centres: ['Oxford', 'Sheffield', 'Hounslow', 'Nottingham', 'Leeds']


- 11002.0: "Oxford"
- 11014.0: "Sheffield"
- 11018.0: "Hounslow"
- 11013.0: "Nottingham"
- 11010.0: "Leeds"

In [41]:
to_remove = [11002.0,11014.0,11018.0,11013.0,11010.0]

In [42]:
df_expo_0_int = df_expo_0[~(df_expo_0['Assessment_centre'].isin(to_remove))].sort_values(by=['eid'])
df_expo_0_ext = df_expo_0[(df_expo_0['Assessment_centre'].isin(to_remove))].sort_values(by=['eid'])

In [43]:
data_final_0_int = data_final_0[data_final_0.index.isin(df_expo_0_int.index)].sort_values(by=['eid'])
data_final_0_ext = data_final_0[data_final_0.index.isin(df_expo_0_ext.index)].sort_values(by=['eid'])

In [44]:
del df_expo_0_int['Assessment_centre']
del df_expo_0_ext['Assessment_centre']
del df_expo_1['Assessment_centre']

In [45]:
# Drop constant training columns
drop_col = []
for col in df_expo_0_int.columns:
    if len(df_expo_0_int[col].value_counts())==1:
        drop_col.append(col)

df0 = df_expo_0_int.drop(drop_col, axis=1)
df1 = df_expo_0_ext.drop(drop_col, axis=1)
df2 = df_expo_1.drop(drop_col, axis=1)

In [46]:
sparseFeatures = set()
missingFeatures = pd.concat([df0]).isna().sum().divide(df0.shape[0]).sort_values(ascending=False)

for name in missingFeatures.sort_values(ascending=False)[missingFeatures>0.3].keys():
    sparseFeatures.add(name)

df0 = df0.drop(sparseFeatures, axis=1)
df1 = df1.drop(sparseFeatures, axis=1)
df2 = df2.drop(sparseFeatures, axis=1)

missingPatients = pd.concat([df0, df1, df2]).isna().sum(axis=1).divide(df0.shape[1]).sort_values(ascending=False)
sparsePatients = missingPatients[missingPatients > 0.3].index

trainCohort_X = df0[~df0.index.isin(sparsePatients)]
extCohort_X = df1[~df1.index.isin(sparsePatients)]
traceCohort_X = df2[~df2.index.isin(sparsePatients)]

In [47]:
trainCohort_X.shape, extCohort_X.shape, traceCohort_X.shape

((73344, 173), (31444, 173), (3967, 173))

In [48]:
data_final_0_int.shape, data_final_0_ext.shape, data_final_1.shape

((73632, 8), (31556, 8), (3969, 8))

In [49]:
trainCohort_Y = data_final_0_int[data_final_0_int.index.isin(trainCohort_X.index)].sort_values(by=['eid'])
extCohort_Y = data_final_0_ext[data_final_0_ext.index.isin(extCohort_X.index)].sort_values(by=['eid'])
traceCohort_Y = data_final_1[data_final_1.index.isin(traceCohort_X.index)].sort_values(by=['eid'])

In [50]:
trainCohort_X.shape, extCohort_X.shape, traceCohort_X.shape

((73344, 173), (31444, 173), (3967, 173))

In [51]:
trainCohort_Y.shape, extCohort_Y.shape, traceCohort_Y.shape

((73344, 8), (31444, 8), (3967, 8))

## CHAPTER 1: 

In [52]:
trainCohort_X1 = trainCohort_X
extCohort_X1 = extCohort_X
traceCohort_X1 = traceCohort_X

trainCohort_Y1 = trainCohort_Y
extCohort_Y1 = extCohort_Y
traceCohort_Y1 = traceCohort_Y 

In [53]:
dfs_X = [trainCohort_X1, extCohort_X1, traceCohort_X1]
dfs_Y = [trainCohort_Y1, extCohort_Y1, traceCohort_Y1] 

for x in range(0,len(dfs_X)):
    dfs_X[x]['PastDiabetes'] = dfs_Y[x]['PastDiabetes']
    dfs_X[x]['PastCVD'] = dfs_Y[x]['PastCVD']

/var/folders/yf/w9tkbs3512s2h935hh0_xhpr0000gn/T/ipykernel_4738/823693964.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_X[x]['PastDiabetes'] = dfs_Y[x]['PastDiabetes']
/var/folders/yf/w9tkbs3512s2h935hh0_xhpr0000gn/T/ipykernel_4738/823693964.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_X[x]['PastCVD'] = dfs_Y[x]['PastCVD']


In [54]:
DropCols = ['CVD',
            'Depression',
            'Diabetes',
            'PastCVD',
            'PastDiabetes',
#            'target',
            'visit0',
            'visit1']

In [55]:
# Drop the columns from the dataframes
trainCohort_Y1 = trainCohort_Y1.drop(columns=[col for col in DropCols if col in trainCohort_Y1.columns])
extCohort_Y1 = extCohort_Y1.drop(columns=[col for col in DropCols if col in extCohort_Y1.columns])
traceCohort_Y1 = traceCohort_Y1.drop(columns=[col for col in DropCols if col in traceCohort_Y1.columns])

In [56]:
trainCohort_X1.shape, trainCohort_Y1.shape

((73344, 175), (73344, 1))

In [57]:
extCohort_X1.shape, extCohort_Y1.shape

((31444, 175), (31444, 1))

In [58]:
traceCohort_X1.shape, traceCohort_Y1.shape

((3967, 175), (3967, 1))

In [59]:
PAT = '/Users/marinacamacho/Desktop/Master_I/Data_Final'
DIS = "Depression_1"

In [60]:
with open(f"{PAT}/trainCohort_{DIS}", "wb") as fp:
    pickle.dump(trainCohort_X1, fp)
with open(f"{PAT}/trainLabels_{DIS}", "wb") as fp:
    pickle.dump(trainCohort_Y1, fp)

with open(f"{PAT}/externalCohort_{DIS}", "wb") as fp:
    pickle.dump(extCohort_X1, fp)
with open(f"{PAT}/externalLabels_{DIS}", "wb") as fp:
    pickle.dump(extCohort_Y1, fp)

with open(f"{PAT}/traceCohort_{DIS}", "wb") as fp:
    pickle.dump(traceCohort_X1, fp)
with open(f"{PAT}/traceLabels_{DIS}", "wb") as fp:
    pickle.dump(traceCohort_Y1, fp)

In [61]:
lista = list(trainCohort_X1[trainCohort_X1['PastCVD']==1].index)
PastCVD = trainCohort_Y1[trainCohort_Y1.index.isin(lista)]

In [62]:
PastCVD['target'].value_counts()

0    12453
1     1029
Name: target, dtype: int64

In [63]:
trainCohort_Y1['target'].value_counts()

0    58407
1    14937
Name: target, dtype: int64

## CHAPTER 2: 

In [64]:
trainCohort_X2 = trainCohort_X
extCohort_X2 = extCohort_X
traceCohort_X2 = traceCohort_X

trainCohort_Y2 = trainCohort_Y
extCohort_Y2 = extCohort_Y
traceCohort_Y2 = traceCohort_Y

In [65]:
dfs_X = [trainCohort_X2, extCohort_X2, traceCohort_X2]
dfs_Y = [trainCohort_Y2, extCohort_Y2, traceCohort_Y2] 

In [66]:
for x in range(0,len(dfs_X)):
    #dfs_X[x] = dfs_X[x][dfs_X[x].index.isin(dfs_Y[x][dfs_Y[x]['PastCVD'] == 1].index )]
    #dfs_Y[x] = dfs_Y[x][dfs_Y[x].index.isin(dfs_Y[x][dfs_Y[x]['PastCVD'] == 1].index )]
    dfs_X[x] = dfs_X[x][dfs_X[x].index.isin(dfs_Y[x][dfs_Y[x]['PastDiabetes'] == 1].index )]
    dfs_Y[x] = dfs_Y[x][dfs_Y[x].index.isin(dfs_Y[x][dfs_Y[x]['PastDiabetes'] == 1].index )]

In [67]:
trainCohort_X2.shape, extCohort_X2.shape, traceCohort_X2.shape
#((73344, 175), (31444, 175), (3967, 175))

((73344, 175), (31444, 175), (3967, 175))

In [68]:
trainCohort_X2 = dfs_X[0]
extCohort_X2 = dfs_X[1]
traceCohort_X2 = dfs_X[2]

trainCohort_Y2 = dfs_Y[0]
extCohort_Y2 = dfs_Y[1]
traceCohort_Y2 = dfs_Y[2]

In [69]:
trainCohort_X2.shape, extCohort_X2.shape, traceCohort_X2.shape
#((13482, 175), (5619, 175), (682, 175))
#((5874, 175), (2379, 175), (249, 175))

((5874, 175), (2379, 175), (249, 175))

In [70]:
trainCohort_Y2.shape, extCohort_Y2.shape, traceCohort_Y2.shape
#((13482, 8), (5619, 8), (682, 8))
#((5874, 8), (2379, 8), (249, 8))

((5874, 8), (2379, 8), (249, 8))

In [71]:
DropCols = ['CVD',
            'Depression',
            'Diabetes',
            'PastCVD',
            'PastDiabetes',
#            'target',
            'visit0',
            'visit1']

In [72]:
# Drop the columns from the dataframes
trainCohort_Y2 = trainCohort_Y2.drop(columns=[col for col in DropCols if col in trainCohort_Y2.columns])
extCohort_Y2 = extCohort_Y2.drop(columns=[col for col in DropCols if col in extCohort_Y2.columns])
traceCohort_Y2 = traceCohort_Y2.drop(columns=[col for col in DropCols if col in traceCohort_Y2.columns])

In [73]:
DropCols = ['PastDiabetes']
#DropCols = ['PastCVD']

In [74]:
# Drop the columns from the dataframes
trainCohort_X2 = trainCohort_X2.drop(columns=[col for col in DropCols if col in trainCohort_X2.columns])
extCohort_X2 = extCohort_X2.drop(columns=[col for col in DropCols if col in extCohort_X2.columns])
traceCohort_X2 = traceCohort_X2.drop(columns=[col for col in DropCols if col in traceCohort_X2.columns])

In [75]:
PAT = '/Users/marinacamacho/Desktop/Master_I/Data_Final'
DIS = "Diabetes_Depression_2"
#DIS = "CVD_Depression_2"

In [76]:
with open(f"{PAT}/trainCohort_{DIS}", "wb") as fp:
    pickle.dump(trainCohort_X2, fp)
with open(f"{PAT}/trainLabels_{DIS}", "wb") as fp:
    pickle.dump(trainCohort_Y2, fp)

with open(f"{PAT}/externalCohort_{DIS}", "wb") as fp:
    pickle.dump(extCohort_X2, fp)
with open(f"{PAT}/externalLabels_{DIS}", "wb") as fp:
    pickle.dump(extCohort_Y2, fp)

with open(f"{PAT}/traceCohort_{DIS}", "wb") as fp:
    pickle.dump(traceCohort_X2, fp)
with open(f"{PAT}/traceLabels_{DIS}", "wb") as fp:
    pickle.dump(traceCohort_Y2, fp)

## CHAPTER 3: 

In [77]:
trainCohort_X3 = trainCohort_X
extCohort_X3 = extCohort_X
traceCohort_X3 = traceCohort_X

trainCohort_Y3 = trainCohort_Y
extCohort_Y3 = extCohort_Y
traceCohort_Y3 = traceCohort_Y 

In [78]:
dfs_X = [trainCohort_X3, extCohort_X3, traceCohort_X3]
dfs_Y = [trainCohort_Y3, extCohort_Y3, traceCohort_Y3] 

for x in range(0,len(dfs_X)):
    dfs_X[x]['PastDiabetes'] = dfs_Y[x]['PastDiabetes']
    dfs_X[x]['PastCVD'] = dfs_Y[x]['PastCVD']

/var/folders/yf/w9tkbs3512s2h935hh0_xhpr0000gn/T/ipykernel_4738/2893240587.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_X[x]['PastDiabetes'] = dfs_Y[x]['PastDiabetes']
/var/folders/yf/w9tkbs3512s2h935hh0_xhpr0000gn/T/ipykernel_4738/2893240587.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_X[x]['PastCVD'] = dfs_Y[x]['PastCVD']


In [79]:
DropCols = ['CVD',
#            'Depression',
            'Diabetes',
            'PastCVD',
            'PastDiabetes',
#            'target',
#            'visit0',
            'visit1']
# Drop the columns from the dataframes
trainCohort_Y3 = trainCohort_Y3.drop(columns=[col for col in DropCols if col in trainCohort_Y3.columns])
extCohort_Y3 = extCohort_Y3.drop(columns=[col for col in DropCols if col in extCohort_Y3.columns])

In [80]:
DropCols = ['CVD',
#            'Depression',
            'Diabetes',
            'PastCVD',
            'PastDiabetes',
#            'target',
            'visit0',
#            'visit1'
           ]
# Drop the columns from the dataframes
traceCohort_Y3 = traceCohort_Y3.drop(columns=[col for col in DropCols if col in traceCohort_Y3.columns])

In [81]:
PAT = '/Users/marinacamacho/Desktop/Master_I/Data_Final'
DIS = "Depression_3"

In [82]:
with open(f"{PAT}/trainCohort_{DIS}", "wb") as fp:
    pickle.dump(trainCohort_X3, fp)
with open(f"{PAT}/trainLabels_{DIS}", "wb") as fp:
    pickle.dump(trainCohort_Y3, fp)

with open(f"{PAT}/externalCohort_{DIS}", "wb") as fp:
    pickle.dump(extCohort_X3, fp)
with open(f"{PAT}/externalLabels_{DIS}", "wb") as fp:
    pickle.dump(extCohort_Y3, fp)

with open(f"{PAT}/traceCohort_{DIS}", "wb") as fp:
    pickle.dump(traceCohort_X3, fp)
with open(f"{PAT}/traceLabels_{DIS}", "wb") as fp:
    pickle.dump(traceCohort_Y3, fp)

## CHAPTER 4: 

In [83]:
DropCols = [#"Ethnic", "Skin_colour",#"Assessment_centre",#"Sex",
            #"Recurrent_moderate_depression","Recurrent_severe_depression", "Single_depression_episode",
            "Depressed_2weeks", "Ever_depressed_1week", "Longest_depression", "Number_depression",
            #'APOA', 'APOB','Cholesterol', 'CRP', 'Glucose', 'HbA1c', 'HDL', 'IGF_1',
            #'Testosterone', 'Triglyceride', 
            'Seen_doctor', 'Seen_sychiatrist', 'Body_fat-free_mass','Body_fat_percentage',
            'Assessment_centre']

In [84]:
data4 = pd.read_csv("/Users/marinacamacho/Desktop/Master_I/Participants_Selection/wide_diseases_dates2.csv",delimiter="\,")
data4 = data4.set_index('eid')
data4 = data4.sort_index(axis=1)

/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [85]:
healthy4 = pd.read_csv("/Users/marinacamacho/Desktop/Master_I/Participants_Selection/Healthy_Participants.csv",delimiter="\,")
healthy4 = healthy4.set_index('eid')
healthy4 = healthy4.sort_index(axis=1)
dates4 = pd.read_csv("/Users/marinacamacho/Desktop/Master_I/Participants_Selection/Date_Centre.csv",delimiter="\,")
dates4 = dates4.set_index('eid')
dates4 = dates4.sort_index(axis=1)
healthy4 = healthy4.merge(dates4, left_index=True, right_index=True, how='inner')
healthy4 = healthy4.dropna()
healthy4['CVD'] = None
healthy4['Depression'] = None
healthy4['Diabetes'] = None
healthy4['visit0'] = healthy4['date_centre_0']
healthy4['visit1'] = healthy4['date_centre_1']
del healthy4['date_centre_0']
del healthy4['date_centre_1']

In [86]:
data4_Y = pd.concat([data4,healthy4],axis=0).sort_index(axis=1)

In [87]:
for column in list(data4_Y.columns):
        data4_Y[column] = pd.to_datetime(data4_Y[column])

In [88]:
data4_Y.head()

,CVD,Depression,Diabetes,visit0,visit1
eid,,,,,
1004849,NaT,2019-08-29,1999-10-14,2008-06-20,2013-02-04
1006558,NaT,2014-09-01,NaT,2007-12-17,2013-01-30
1008230,NaT,1998-05-26,NaT,2007-06-25,2012-08-31
1008381,NaT,NaT,2011-01-10,2008-06-09,2012-11-01
1011321,NaT,2013-04-03,NaT,2008-03-03,2013-05-25


In [89]:
# Define function to get the required column value
def get_time0(row):
    conditions = [
        row['Depression'] < row['visit0'],
        row['CVD'] < row['visit0'],
        row['Diabetes'] < row['visit0']
    ]

    if all(conditions):
        return 'Depression-Diabetes-CVD'
    elif conditions[0] and conditions[1]:
        return 'Depression-CVD'
    elif conditions[0] and conditions[2]:
        return 'Depression-Diabetes'
    elif conditions[1] and conditions[2]:
        return 'CVD-Diabetes'
    elif conditions[0]:
        return 'Depression'
    elif conditions[1]:
        return 'CVD'    
    elif conditions[2]:
        return 'Diabetes'
    else:
        return '-'

In [90]:
# Define function to get the required column value
def get_time1(row):
    conditions = [
        row['Depression'] >= row['visit0'],
        row['CVD'] >= row['visit0'],
        row['Diabetes'] >= row['visit0'],
        row['Depression'] <= row['visit1'],
        row['CVD'] <= row['visit1'],
        row['Diabetes'] <= row['visit1'],
    ]

    if all(conditions):
        return 'Depression-Diabetes-CVD'
    elif conditions[0] and conditions[1] and conditions[3] and conditions[4]:
        return 'Depression-CVD'
    elif conditions[0] and conditions[2] and conditions[3] and conditions[5]:
        return 'Depression-Diabetes'
    elif conditions[1] and conditions[2] and conditions[4] and conditions[5]:
        return 'CVD-Diabetes'
    elif conditions[0] and conditions[3]:
        return 'Depression'
    elif conditions[1] and conditions[4]:
        return 'CVD'    
    elif conditions[2] and conditions[5]:
        return 'Diabetes'
    else:
        return '-'

In [91]:
# Define function to get the required column value
def get_time2(row):
    conditions = [
        row['Depression'] > row['visit1'],
        row['CVD'] > row['visit1'],
        row['Diabetes'] > row['visit1']
    ]

    if all(conditions):
        return 'Depression-Diabetes-CVD'
    elif conditions[0] and conditions[1]:
        return 'Depression-CVD'
    elif conditions[0] and conditions[2]:
        return 'Depression-Diabetes'
    elif conditions[1] and conditions[2]:
        return 'CVD-Diabetes'
    elif conditions[0]:
        return 'Depression'
    elif conditions[1]:
        return 'CVD'
    elif conditions[2]:
        return 'Diabetes'
    else:
        return '-'

In [92]:
data4_Y['Time0'] = data4_Y.apply(get_time0, axis=1)
data4_Y['Time1'] = data4_Y.apply(get_time1, axis=1)
data4_Y['Time2'] = data4_Y.apply(get_time2, axis=1)

In [93]:
# Exposome data
df_expo4 = pd.read_csv(f'{LOAD}/Time_0/ext_exposome_0.csv', usecols=lambda col: col not in DropCols)
df_expo4 = df_expo4.set_index('f.eid')
df_expo4.index.name = 'eid'

In [94]:
data4_X = df_expo4[df_expo4.index.isin(data4_Y.index)].sort_values(by=['eid'])
data4_Y = data4_Y[data4_Y.index.isin(data4_X.index)].sort_values(by=['eid'])

In [95]:
# Step 1: Remove constant columns
data4_X = data4_X.loc[:, data4_X.nunique() > 1]

# Step 2: Remove rows with more than 30% missing values
threshold_row = 0.7 * len(data4_X.columns)
data4_X = data4_X.dropna(thresh=threshold_row)

# Step 3: Remove columns with more than 30% missing values
threshold_col = 0.7 * len(data4_X)
data4_X = data4_X.dropna(axis=1, thresh=threshold_col)

In [96]:
data4_X.shape, data4_Y.shape

((6591, 166), (6718, 8))

In [97]:
PAT = '/Users/marinacamacho/Desktop/Master_I/Data_Final'
DIS = "Depression_4"

In [98]:
with open(f"{PAT}/trainCohort_{DIS}", "wb") as fp:
    pickle.dump(data4_X, fp)
with open(f"{PAT}/trainLabels_{DIS}", "wb") as fp:
    pickle.dump(data4_Y, fp)

In [99]:
with open(f"{PAT}/trainLabels_{DIS}", "rb") as fp:
    data4_Y = pickle.load(fp)

In [107]:
earlycause = data4_X.iloc[:,:3]

In [109]:
alluvia = pd.concat([data4_Y,earlycause], axis=1)
columns_to_check = ['Breastfed_baby', 'Adopted_child', 'Maternal_smoking_around_birth']
alluvia = alluvia.dropna(subset=columns_to_check)

In [110]:
alluvia.to_csv(r'/Users/marinacamacho/Desktop/Master_I/Modelling/Alluvia_data4.csv', index = True, header=True)

In [9]:
pwd

'/Users/marinacamacho/Desktop/Master_I/Modelling'

## CHAPTER 5: 

In [91]:
data5_X = data4_X
data5_Y = data4_Y

In [92]:
data5_Y['Breastfed baby'] = data5_X['Breastfed_baby']
data5_Y['Adopted child'] = data5_X['Adopted_child']
data5_Y['Maternal smoking around birth'] = data5_X['Maternal_smoking_around_birth']

In [93]:
del data5_X['Breastfed_baby']
del data5_X['Adopted_child']
del data5_X['Maternal_smoking_around_birth']

In [94]:
PAT = '/Users/marinacamacho/Desktop/Master_I/Data_Final'
DIS = "Depression_5"
with open(f"{PAT}/trainCohort_{DIS}", "wb") as fp:
    pickle.dump(data5_X, fp)
with open(f"{PAT}/trainLabels_{DIS}", "wb") as fp:
    pickle.dump(data5_Y, fp)

In [95]:
list(data4_X.columns)

['Sex',
 'Waist_circumference',
 'Hip_circumference',
 'Sleep_duration',
 'Nap_during_day',
 'Sleeplessness',
 'Daytime_dozing/sleeping',
 'Current_smoking',
 'Past_smoking',
 'Cooked_vegetable_intake',
 'Salad/raw_vegetable_intake',
 'Fresh_fruit_intake',
 'Dried_fruit_intake',
 'Oily_fish_intake',
 'Non_oily_fish_intake',
 'Processed_meat_intake',
 'Poultry_intake',
 'Beef_intake',
 'Lamb/mutton_intake',
 'Pork_intake',
 'Cheese_intake',
 'Bread_intake',
 'Cereal_intake',
 'Salt_added',
 'Tea_intake',
 'Water_intake',
 'Dietary_changes_5years',
 'Variation_diet',
 'Unenthusiasm_2weeks',
 'Tenseness_2weeks',
 'Tiredness_2weeks',
 'Drinker_status',
 'Ethnic',
 'BMI',
 'White',
 'Brown',
 'WholeMealgrain',
 'OtherBread',
 'Full_cream',
 'Semi-skimmed',
 'Skimmed',
 'Soya',
 'Other_type_of_milk',
 'Never/rarely_have_milk',
 'Butter/spreadable_butter',
 'Flora_Pro-Active/Benecol',
 'Other_type_of_spread/margarine',
 'Never/rarely_use_spread',
 'Bran_cereal',
 'Biscuit_cereal',
 'Oat_cerea